# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


In [18]:
# Use this cell to write your code for Task 1
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
house_sales = pd.read_csv('house_sales.csv')
missing_city = house_sales['city'].replace('--', np.nan).isna().sum()
print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [19]:
# Use this cell to write your code for Task 2}
# Limpieza de datos 
def clean_dataset(house_sales):
    house_sales_copy = house_sales.copy()
    
    # Columna categórica: city
    house_sales_copy['city'] = house_sales_copy['city'].replace('--', np.nan)
    house_sales_copy['city'] = house_sales_copy['city'].fillna('Unknown')
    
    # Columna numérica objetivo
    house_sales_copy = house_sales_copy[house_sales_copy['sale_price'].notna()]
    house_sales_copy = house_sales_copy[house_sales_copy['sale_price'] >= 0]
    
    # Fechas
    house_sales_copy['sale_date'] = pd.to_datetime(
        house_sales_copy['sale_date'], errors='coerce'
    ).fillna(pd.Timestamp('2023-01-01'))
    
    # Bedrooms
    mean_bedrooms = house_sales_copy.loc[
        house_sales_copy['bedrooms'] >= 0, 'bedrooms'
    ].dropna().mean()
    house_sales_copy['bedrooms'] = house_sales_copy['bedrooms'].fillna(round(mean_bedrooms))
    
    # Months listed
    house_sales_copy['months_listed'] = house_sales_copy['months_listed'].round(1)
    mean_months = house_sales_copy['months_listed'].mean()
    house_sales_copy['months_listed'] = house_sales_copy['months_listed'].fillna(round(mean_months, 1))
    
    # Area
    house_sales_copy['area'] = (
    pd.to_numeric(house_sales_copy['area'].astype(str).str.replace(' sq.m.', '', regex=False), errors='coerce')
    .round(1))
    house_sales_copy['area'].fillna(house_sales_copy['area'].mean().round(1), inplace=True)

    # House type (ordinal)
    house_sales_copy['house_type'] = house_sales_copy['house_type'].replace({
        'Det.': 'Detached', 
        'Semi': 'Semi-detached', 
        'Terr.': 'Terraced'
    })
    house_sales_copy['house_type'] = house_sales_copy['house_type'].fillna(
        house_sales_copy['house_type'].mode()[0]
    )
    
    return house_sales_copy


clean_data = clean_dataset(house_sales)
print(clean_data.dtypes)
print(clean_data.isna().sum())


house_id                  int64
city                     object
sale_price                int64
sale_date        datetime64[ns]
months_listed           float64
bedrooms                  int64
house_type               object
area                    float64
dtype: object
house_id         0
city             0
sale_price       0
sale_date        0
months_listed    0
bedrooms         0
house_type       0
area             0
dtype: int64


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [20]:
# Use this cell to write your code for Task 3
house_sales = pd.read_csv('house_sales.csv')
cleaned_df = clean_dataset(house_sales.copy())
print(cleaned_df.isna().sum())
price_by_rooms = house_sales.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).rename(columns={'mean':'avg_price', 'var':'var_price'}).round(1).reset_index()
print(price_by_rooms)

house_id         0
city             0
sale_price       0
sale_date        0
months_listed    0
bedrooms         0
house_type       0
area             0
dtype: int64
   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [21]:
# Use this cell to write your code for Task 4
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
train_data = pd.read_csv("train.csv")
val_data   = pd.read_csv("validation.csv")

# Dummies
train_dummies = pd.get_dummies(train_data, drop_first=True)
val_dummies   = pd.get_dummies(val_data, drop_first=True)

# Alinear columnas
train_dummies, val_dummies = train_dummies.align(val_dummies, join="left", axis=1, fill_value=0)

# Features y target
X_train = train_dummies.drop("sale_price", axis=1).values
y_train = train_dummies["sale_price"].values
X_val   = val_dummies.drop("sale_price", axis=1, errors="ignore").values

# Entrenar modelo baseline
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_val)

# Resultado baseline
base_result = pd.DataFrame({
    "house_id": val_data["house_id"],
    "price": y_pred
})
X_train

array([[1.634561e+06, 7.000000e+00, 6.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.009770e+06, 8.100000e+00, 6.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.946667e+06, 5.400000e+00, 5.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [1.919273e+06, 9.100000e+00, 5.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.512003e+06, 6.100000e+00, 4.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.452168e+06, 7.600000e+00, 5.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00]])

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [22]:
# Use this cell to write your code for Task 5
ridge = Ridge(alpha=1000) 
ridge.fit(X_train, y_train) 
y_pred_ridge = ridge.predict(X_val) 
compare_result = pd.DataFrame({ "house_id": val_data["house_id"], "price":
                                y_pred_ridge}) 
print(compare_result)

     house_id          price
0     1331375  125096.521557
1     1630115  263759.407449
2     1645745  383222.562414
3     1336775  120296.254258
4     1888274  254263.290262
..        ...            ...
295   1986255  344159.311895
296   1896276  363680.414200
297   1758223  238675.015131
298   1752010  177203.142992
299   1651404  359672.536308

[300 rows x 2 columns]


# Analisis complementario
Analizar qué características influyen más en el precio de venta predicho por el modelo Ridge.

In [23]:
# Task 5 — Importancia de variables
import pandas as pd
import numpy as np

# Calcular la importancia de cada variable en el modelo Ridge
importance = pd.Series(
    ridge.coef_,
    index=train_dummies.drop("sale_price", axis=1).columns
).sort_values(key=abs, ascending=False)

# Mostrar las 10 variables más influyentes
print("Top 10 variables que más impactan en el precio de venta:\n")
print(importance.head(10))

Top 10 variables que más impactan en el precio de venta:

city_Teasdale               10086.110359
city_Riverford              -6933.533688
house_type_Terraced         -3516.640831
city_Silvertown              3393.866249
area                          759.689206
house_type_Semi-detached     -501.175897
sale_date_2020-04-14         -211.907720
sale_date_2020-08-16         -154.865178
sale_date_2022-01-17         -146.260670
sale_date_2020-09-21         -143.405649
dtype: float64
